0. Import bibliotek

In [44]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, TIMESTAMP, Float, Boolean, BigInteger, Date
import psycopg2
import sys
from psycopg2 import OperationalError, errorcodes, errors


1. Stworzenie bazy danych

In [10]:
connection_parameters = { 
    'host': 'localhost',
    'port': 5432,
    'user' : 'postgres',
    'password' : 'postgres'
    }

In [11]:
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()

    # get the line number when exception occured
    line_n = traceback.tb_lineno

    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type)

    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)

    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

In [12]:
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successful..................")
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)

        # set the connection to 'None' in case of error
        conn = None
    
    return conn

In [14]:
conn = connect(connection_parameters)
conn.autocommit = True

if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping database irisdb if exists
        cursor.execute("DROP DATABASE IF EXISTS MovieRecommenderdb;")
    
        # Creating a database
        cursor.execute("CREATE DATABASE MovieRecommenderdb;");
        print("MovieRecommenderdb database is created successfully..................")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
MovieRecommenderdb database is created successfully..................


1. Wczytanie danych

In [61]:
path = "C:\\Users\\dabro\\Desktop\\Studia\\INZ\\data\\transfromed_data\\"

In [62]:
df_keywords = pd.read_csv(path + "keywords.csv")
df_keywords.head()

,movieId,keywords
0,862,"['jealousy', 'toy', 'boy', 'friendship', 'frie..."
1,8844,"['board game', 'disappearance', ""based on chil..."
2,15602,"['fishing', 'best friend', 'duringcreditssting..."
3,31357,"['based on novel', 'interracial relationship',..."
4,11862,"['baby', 'midlife crisis', 'confidence', 'agin..."


In [63]:

df_credits = pd.read_csv(path + "credits.csv")
df_credits.head()

,cast,director,movieId
0,"['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...",['John Lasseter'],862
1,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D...",['Joe Johnston'],8844
2,"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret...",['Howard Deutch'],15602
3,"['Whitney Houston', 'Angela Bassett', 'Loretta...",['Forest Whitaker'],31357
4,"['Steve Martin', 'Diane Keaton', 'Martin Short...",['Charles Shyer'],11862


In [64]:
df_movies = pd.read_csv(path + "movies_metadata.csv")
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,movieId,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,IMBD_poster_path,IMBD_backdrop_path
0,False,Toy Story Collection,30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,['English'],Released,NaN,Toy Story,False,7.7,5415.0,https://image.tmdb.org/t/p/w500//7G9915LfUQ2lV...,https://image.tmdb.org/t/p/w500//9FBwqcd9IRruE...
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"['English', 'Français']",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN
2,False,Grumpy Old Men Collection,0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,['English'],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,https://image.tmdb.org/t/p/w500//nLvUdqgPgm3F8...,https://image.tmdb.org/t/p/w500//hypTnLot2z8wp...
3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,['English'],Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,NaN
4,False,Father of the Bride Collection,0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,['English'],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,https://image.tmdb.org/t/p/w500//nts4iOmNnq7GN...,https://image.tmdb.org/t/p/w500//7qwE57OVZmMJC...


In [65]:
#ratings
df_ratings = pd.read_csv(path + "ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp
0,11,32,3.5,1231677006
1,11,34,4.0,1231677071
2,11,47,3.5,1231677031
3,11,110,3.5,1231676989
4,11,165,3.5,1231677046


In [66]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42277 entries, 0 to 42276
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  42277 non-null  bool   
 1   belongs_to_collection  4358 non-null   object 
 2   budget                 42277 non-null  int64  
 3   genres                 42277 non-null  object 
 4   homepage               7241 non-null   object 
 5   movieId                42277 non-null  int64  
 6   imdb_id                42261 non-null  object 
 7   original_language      42266 non-null  object 
 8   original_title         42277 non-null  object 
 9   overview               41362 non-null  object 
 10  popularity             42277 non-null  float64
 11  poster_path            41906 non-null  object 
 12  production_companies   42277 non-null  object 
 13  production_countries   42277 non-null  object 
 14  release_date           42196 non-null  object 
 15  re

3. Stworzenie tabel w bazie danych

In [68]:
host= "localhost"
database="movierecommenderdb"
user= connection_parameters['user']
password= connection_parameters['password']
port= "5432"


engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Define the table's architecture
metadata = MetaData()

credits_table = Table(
    'Credits',
    metadata,
    Column('movieId', Integer, primary_key=True),  # Primary key
    Column('director', String),
    Column('cast', String),
    # Add more columns and data types as needed
)

keywords_table = Table(
    'Keywords',
    metadata,
    Column('movieId', Integer, primary_key=True),  # Primary key
    Column('keywords', String),
    # Add more columns and data types as needed
)

movies_table = Table(
    'Movies',
    metadata,
    Column('movieId', BigInteger, primary_key=True),  # Primary key
    Column('title', String),
    Column('genres', String),
    Column('overview', String),
    Column('release_date', Date),
    Column('runtime', Float),
    Column('vote_average', Float),
    Column('vote_count', Float),
    Column('popularity', Float),
    Column('budget', Integer),
    Column('revenue', Float),
    Column('production_companies', String),
    Column('production_countries', String),
    Column('spoken_languages', String),
    Column('status', String),
    Column('tagline', String),
    Column('original_language', String),
    Column('original_title', String),
    Column('imdb_id', String),
    Column('adult', Boolean),
    Column('video', Boolean),
    Column('homepage', String),
    Column('belongs_to_collection', String),
    Column('IMBD_poster_path', String),
    Column('IMBD_backdrop_path', String),
    Column('poster_path', String),
    # Add more columns and data types as needed
)

ratings_table = Table(
    'Ratings',
    metadata,
    Column('userId', Integer, primary_key=True),  # Primary key
    Column('movieId', Integer),
    Column('rating', Float),
    Column('timestamp', TIMESTAMP),
    # Add more columns and data types as needed
)


# Create the table in the database
metadata.create_all(engine)

df_credits.to_sql('Credits', engine,if_exists='replace', index=False)
df_keywords.to_sql('Keywords', engine,if_exists='replace', index=False)
df_movies.to_sql('Movies', engine,if_exists='replace', index=False)
df_ratings.to_sql('Ratings', engine,if_exists='replace', index=False)

print("Data is stored in PostgreSQL database successfully..................")

Data is stored in PostgreSQL database successfully..................
